In [1]:
import gc
import os
import pickle
import random
import joblib
import pandas as pd
# import polars as pd
from tqdm import tqdm

import numpy as np
import torch
from torch.utils.data import TensorDataset, Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from sklearn.model_selection import StratifiedKFold
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.callbacks.lr_monitor import LearningRateMonitor
import torch_geometric.nn as pyg_nn
from sklearn.metrics import average_precision_score


In [2]:
class CFG:
    PREPROCESS = True
    SHRINKING = True
    SHRINKING_SIZE = 0.01
    EPOCHS = 20 #20
    BATCH_SIZE = 4096
    LR = 1e-4
    WD = 1e-6

    NBR_FOLDS = 2
    SELECTED_FOLDS = [0]

    SEED = 42

In [3]:
# import tensorflow as tf
import torch
def set_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    #tf.random.set_seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)

set_seeds(seed=CFG.SEED)

In [4]:
# import pandas as pd
# import numpy as np
# from rdkit import Chem
# from torch_geometric.data import Data
# import torch
# from joblib import Parallel, delayed
# from tqdm import tqdm

# # 读取训练数据
# train_raw = pd.read_parquet('data/train.parquet')
# # 如果开启了 SHRINKING，按照给定的比例采样数据
# if CFG.SHRINKING:
#     # 按 'molecule_smiles' 分组，进行采样
#     molecules = train_raw['molecule_smiles'].unique()
#     sampled_molecules = np.random.choice(molecules, size=int(len(molecules) * CFG.SHRINKING_SIZE), replace=False)
    
#     # 根据采样的分子过滤原数据
#     train_raw = train_raw[train_raw['molecule_smiles'].isin(sampled_molecules)]

# smiles = train_raw[train_raw['protein_name'] == 'BRD4']['molecule_smiles'].values
# train_smile = pd.DataFrame(smiles)

# assert (smiles != train_raw[train_raw['protein_name'] == 'HSA']['molecule_smiles'].values).sum() == 0
# assert (smiles != train_raw[train_raw['protein_name'] == 'sEH']['molecule_smiles'].values).sum() == 0

# train_smile['bind1'] = train_raw[train_raw['protein_name']=='BRD4']['binds'].values
# train_smile['bind2'] = train_raw[train_raw['protein_name']=='HSA']['binds'].values
# train_smile['bind3'] = train_raw[train_raw['protein_name']=='sEH']['binds'].values

# # 将bind1, bind2, bind3三列合并为一个新的列，每个元素是一个长度为3的列表
# train_smile['binds_combined'] = train_smile.apply(
#     lambda row: [row['bind1'], row['bind2'], row['bind3']], axis=1)



In [5]:
# train_smile['binds_combined'][0:10]
# # # 检查 'binds_combined' 列表中是否包含 [0, 1, 0]
# # target = [0, 1, 0]
# # exists = (train_smile['binds_combined'].apply(lambda x: x == target)).any()

# # print(exists)  # 如果存在则返回 True，否则返回 False
# train_smile.head()

In [6]:
import pandas as pd
import numpy as np
from rdkit import Chem
from torch_geometric.data import Data
import torch
from joblib import Parallel, delayed
from tqdm import tqdm

# 定义编码 SMILES 为图结构的函数
def encode_smile(smile):
    graph = smi_to_graph(smile)
    return graph

# 定义分子转图结构的函数
def smi_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    
    # 获取节点特征：这里用原子编号作为特征（你可以根据需要调整）
    atom_features = []
    for atom in mol.GetAtoms():
        atom_features.append(atom.GetAtomicNum())  # 获取原子序数作为特征
    
    # 获取边的连接信息
    edge_index = []
    for bond in mol.GetBonds():
        # bond.GetBeginAtomIdx() 和 bond.GetEndAtomIdx() 返回的是两个原子之间的连接
        edge_index.append([bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()])
        edge_index.append([bond.GetEndAtomIdx(), bond.GetBeginAtomIdx()])
    
    # 转换成 PyTorch Geometric 格式
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    atom_features = torch.tensor(atom_features, dtype=torch.float).view(-1, 1)  # 节点特征：每个原子对应一个特征
    
    return Data(x=atom_features, edge_index=edge_index)

if CFG.PREPROCESS:
    # 读取训练数据
    train_raw = pd.read_parquet('data/train.parquet')
    # 如果开启了 SHRINKING，按照给定的比例采样数据
    if CFG.SHRINKING:
        # 按 'molecule_smiles' 分组，进行采样
        molecules = train_raw['molecule_smiles'].unique()
        sampled_molecules = np.random.choice(molecules, size=int(len(molecules) * CFG.SHRINKING_SIZE), replace=False)
        
        # 根据采样的分子过滤原数据
        train_raw = train_raw[train_raw['molecule_smiles'].isin(sampled_molecules)]


    smiles = train_raw[train_raw['protein_name'] == 'BRD4']['molecule_smiles'].values
    assert (smiles != train_raw[train_raw['protein_name'] == 'HSA']['molecule_smiles'].values).sum() == 0
    assert (smiles != train_raw[train_raw['protein_name'] == 'sEH']['molecule_smiles'].values).sum() == 0

    
    train_smile = pd.DataFrame(smiles)

    train_smile['bind1'] = train_raw[train_raw['protein_name']=='BRD4']['binds'].values
    train_smile['bind2'] = train_raw[train_raw['protein_name']=='HSA']['binds'].values
    train_smile['bind3'] = train_raw[train_raw['protein_name']=='sEH']['binds'].values

    # 将bind1, bind2, bind3三列合并为一个新的列，每个元素是一个长度为3的列表
    train_smile['binds_combined'] = train_smile.apply(
    lambda row: [row['bind1'], row['bind2'], row['bind3']], axis=1)

    # 使用 joblib 并行化处理# 对采样后的数据进行图结构转换
    graphs = Parallel(n_jobs=60)(delayed(encode_smile)(smile) for smile in tqdm(smiles))


    # 提取特征和边
    train = []

    # 在转换图的过程中，直接对每个图数据添加bind_target
    for graph, bind_target in tqdm(zip(graphs, train_smile['binds_combined'])):
        # 创建图数据并保存
        train.append({
            'x': graph.x,              # 节点特征
            'edge_index': graph.edge_index,  # 边连接信息
            'bind': bind_target        # 当前smile的bind信息
        })

    # 保存图数据（可以选择保存为 PyTorch 的 Data 对象）
    torch.save(train, 'output/train_graphs.pt')

    # 对测试集进行同样的处理
    # test_raw = pd.read_parquet('data/test.parquet')
    # smiles = test_raw['molecule_smiles'].values
    # # test_graphs = Parallel(n_jobs=60)(delayed(encode_smile)(smile) for smile in tqdm(smiles))

    # test_graphs = Parallel(n_jobs=60)(delayed(encode_smile)(smile) for smile in tqdm(smiles))
    # 如果开启了 SHRINKING，按照给定的比例采样数据
    # if CFG.SHRINKING:
    #     test_raw = test_raw.sample(frac=CFG.SHRINKING_SIZE, random_state=CFG.SEED)  # 根据 SHRINKING_SIZE 采样数据
    #     smiles = test_raw['molecule_smiles'].values
    #     # 对采样后的数据进行图结构转换
    #     test_graphs = Parallel(n_jobs=60)(delayed(encode_smile)(smile) for smile in tqdm(smiles))
    # else:
    #     # 使用 joblib 并行化处理
    #     test_graphs = Parallel(n_jobs=60)(delayed(encode_smile)(smile) for smile in tqdm(smiles))

    # test = []
    # for graph in test_graphs:
    #     test.append({
    #         'x': graph.x,
    #         'edge_index': graph.edge_index
    #     })

    # torch.save(test, 'output/test_graphs.pt')
    test = torch.load('output/test_graphs.pt')

else:
    train = torch.load('output/train_graphs.pt')
    test = torch.load('output/test_graphs.pt')


100%|██████████| 984156/984156 [11:32<00:00, 1421.60it/s]
984156it [00:01, 831402.85it/s]


In [7]:
from torch_geometric.nn import GCNConv
class MyGNNModel(pl.LightningModule):
    def __init__(self, hidden_dim=6, output_dim=3, lr=1e-3, weight_decay=1e-6):
        super(MyGNNModel, self).__init__()
        self.save_hyperparameters()

        # 图卷积层
        self.conv1 = GCNConv(in_channels=1, out_channels=hidden_dim)  # 第一层GCN
        self.conv2 = GCNConv(in_channels=hidden_dim, out_channels=hidden_dim * 2)  # 第二层GCN
        self.conv3 = GCNConv(in_channels=hidden_dim * 2, out_channels=hidden_dim * 4)  # 第三层GCN

        # 全连接层
        self.fc1 = torch.nn.Linear(hidden_dim * 4, 64)
        self.fc2 = torch.nn.Linear(64, 32)
        self.fc3 = torch.nn.Linear(32, output_dim)

        # Dropout
        self.dropout = torch.nn.Dropout(0.1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        #print(x.shape)
        #print(edge_index.shape)

        # 图卷积操作
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))

        # 全局池化（使用最大池化）
        x = pyg_nn.global_max_pool(x, data.batch)

        # 全连接层
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)

        return x

    def training_step(self, batch, batch_idx):
        # 目标标签
        y = batch.y

        # 前向传播
        logits = self(batch)

        # 计算损失
        loss = F.binary_cross_entropy_with_logits(logits, y)

        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        y = batch.y
        logits = self(batch)
        loss = F.binary_cross_entropy_with_logits(logits, y)
        self.log('val_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr, weight_decay=self.hparams.weight_decay)
        return optimizer

In [8]:
from sklearn.model_selection import train_test_split
from torch_geometric.data import DataLoader
# 使用正常训练
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
FEATURES = [f'enc{i}' for i in range(142)]
TARGETS = ['bind1', 'bind2', 'bind3']
all_preds = []

# # 检查每个graph的bind1, bind2, bind3长度
# for graph in train:
#     print(graph['bind'])
# Assuming `train` is a list of graph data dictionaries
# Here, train is a list of dictionaries where each dictionary contains 'x', 'edge_index', and 'y'
# train_data = [Data(x=graph['x'], edge_index=graph['edge_index'], y=[torch.tensor(graph['bind'], dtype=torch.float32)]) for graph in train]
train_data = [Data(x=graph['x'], edge_index=graph['edge_index'], y=torch.tensor([graph['bind']], dtype=torch.float32)) for graph in train]



# Train-validation split (splitting the list of graphs)
train_data, valid_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Now we can create DataLoader from these lists
train_loader = DataLoader(train_data, batch_size=CFG.BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=CFG.BATCH_SIZE)

# # Convert pandas dataframes to PyTorch tensors
# X_train = torch.tensor(train.loc[train_idx, FEATURES].values, dtype=torch.int)
# y_train = torch.tensor(train.loc[train_idx, TARGETS].values, dtype=torch.float16)
# X_val = torch.tensor(train.loc[valid_idx, FEATURES].values, dtype=torch.int)
# y_val = torch.tensor(train.loc[valid_idx, TARGETS].values, dtype=torch.float16)

# # Create TensorDatasets
# train_dataset = TensorDataset(X_train, y_train)
# valid_dataset = TensorDataset(X_val, y_val)

# # Create DataLoaders
# train_loader = DataLoader(train_dataset, batch_size=CFG.BATCH_SIZE, shuffle=True)
# valid_loader = DataLoader(valid_dataset, batch_size=CFG.BATCH_SIZE)

# Initialize the model
model = MyGNNModel(lr=CFG.LR, weight_decay=CFG.WD)

# Define callbacks
early_stop_callback = EarlyStopping(monitor="val_loss", mode="min", patience=5, verbose=True)
checkpoint_callback = ModelCheckpoint(monitor="val_loss", dirpath="./ckpoint/GNN", filename="model", save_top_k=1, mode="min")
lr_monitor = LearningRateMonitor(logging_interval='epoch')

# Trainer setup
trainer = pl.Trainer(
    max_epochs=CFG.EPOCHS,
    callbacks=[early_stop_callback, checkpoint_callback, lr_monitor],
    devices=1,
    accelerator="cpu",  # Adjust based on your hardware
    enable_progress_bar=True,
)

# Train the model
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=valid_loader)


# Load model onto the GPU
model = MyGNNModel.load_from_checkpoint(checkpoint_callback.best_model_path).to(device)
print("load ok")
model.eval()  # Set the model to evaluation mode

# Perform batched inference with data also on GPU
all_preds = []
all_targets = []

with torch.no_grad():  # Disable gradient computation for inference
    for batch in tqdm(valid_loader, desc="Validation Progress"):
        # Access the graph data in the batch
        batch_x = batch.x.to(device)  # Node features (x)
        # print(batch_x.shape)
        batch_edge_index = batch.edge_index.to(device)  # Edge index (edge_index)
        # print(batch_edge_index.shape)
        batch_y = batch.y.to(device)  # Ground truth labels (y)
        # print(batch_y.shape)

        # Predict for the batch
        preds = model(batch.to(device))  # Model expects x and edge_index

        # Move predictions and targets to CPU for concatenation
        all_preds.append(preds.cpu())
        all_targets.append(batch_y.cpu())

# Concatenate all batches into single tensors
all_preds = torch.cat(all_preds, dim=0)
all_targets = torch.cat(all_targets, dim=0)

# Compute the score
score = average_precision_score(all_targets.numpy(), all_preds.numpy(), average='micro')
print('Validation Score (Average Precision):', score)




d:\jupyter notebook\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
d:\jupyter notebook\Lib\site-packages\pytorch_lightning\trainer\setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


d:\jupyter notebook\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory D:\zyh\bddm\protein_predict\ckpoint\GNN exists and is not empty.

  | Name    | Type    | Params | Mode 
--------------------------------------------
0 | conv1   | GCNConv | 12     | train
1 | conv2   | GCNConv | 84     | train
2 | conv3   | GCNConv | 312    | train
3 | fc1     | Linear  | 1.6 K  | train
4 | fc2     | Linear  | 2.1 K  | train
5 | fc3     | Linear  | 99     | train
6 | dropout | Dropout | 0      | train
--------------------------------------------
4.2 K     Trainable params
0         Non-trainable params
4.2 K     Total params
0.017     Total estimated model params size (MB)
13        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

d:\jupyter notebook\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
d:\jupyter notebook\Lib\site-packages\pytorch_lightning\utilities\data.py:78: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 167803. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
d:\jupyter notebook\Lib\site-packages\pytorch_lightning\utilities\data.py:78: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 167586. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
d:\jupyter notebook\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the val

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 5 records. Best score: 0.034. Signaling Trainer to stop.


load ok


Validation Progress: 100%|██████████| 49/49 [00:03<00:00, 12.35it/s]

Validation Score (Average Precision): 0.006941758895845293


In [9]:
print(all_preds[25])
print(all_targets[25])

tensor([-5.1087, -5.5801, -4.7407])
tensor([0., 0., 0.])


In [10]:
from torch_geometric.data import DataLoader

# 假设 test 数据是一个包含多个 graph 的列表，每个 graph 是一个 Data 对象
test_data = [Data(x=torch.tensor(graph['x'], dtype=torch.float32), 
                  edge_index=torch.tensor(graph['edge_index'], dtype=torch.long)) for graph in test]


# test_dataset = TensorDataset(test_data)
test_loader = DataLoader(test_data,batch_size=CFG.BATCH_SIZE,shuffle=False)

# Perform batched inference
all_preds_test = []

model.eval()  # Set model to evaluation mode

with torch.no_grad():  # Disable gradient computation for inference
    for batch in tqdm(test_loader, desc="Test Data Inference Progress"):
        # Predict for the batch
        preds = model(batch.to(device))
        
        # Move predictions to CPU and append to results
        all_preds_test.append(preds.cpu())  

# Concatenate all predictions into a single tensor
final_preds = torch.cat(all_preds_test, dim=0)

# Min-Max Normalization to scale predictions between 0 and 1
min_val = final_preds.min()
max_val = final_preds.max()
final_preds = (final_preds - min_val) / (max_val - min_val)

# Convert to NumPy for further processing
final_preds = final_preds.numpy()

# Output the final predictions
print("Inference completed. Predictions shape:", final_preds.shape)

C:\Users\PC\AppData\Local\Temp\ipykernel_167416\162507663.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_data = [Data(x=torch.tensor(graph['x'], dtype=torch.float32),
C:\Users\PC\AppData\Local\Temp\ipykernel_167416\162507663.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(graph['edge_index'], dtype=torch.long)) for graph in test]
d:\jupyter notebook\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
Test Data Inference Progress: 100%|██████████| 409/409 [00:35<00:00, 11.36it/s]


Inference completed. Predictions shape: (1674896, 3)


In [11]:
tst = pd.read_parquet('data/test.parquet')
tst['binds'] = 0
tst.loc[tst['protein_name']=='BRD4', 'binds'] = final_preds[(tst['protein_name']=='BRD4').values, 0]
tst.loc[tst['protein_name']=='HSA', 'binds'] = final_preds[(tst['protein_name']=='HSA').values, 1]
tst.loc[tst['protein_name']=='sEH', 'binds'] = final_preds[(tst['protein_name']=='sEH').values, 2]
tst[['id', 'binds']].to_csv('submission_gnn.csv', index = False)

C:\Users\PC\AppData\Local\Temp\ipykernel_167416\1077872100.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.5623296 0.5623296 0.5623296 ... 0.5623296 0.5623296 0.5623296]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tst.loc[tst['protein_name']=='BRD4', 'binds'] = final_preds[(tst['protein_name']=='BRD4').values, 0]
